In [22]:
# Import packages
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import math as ma

In [23]:
#Loading And Preparing Data From YF
ticker = 'BTC-USD'

stock_data = yf.download(ticker, interval = '1d')['Close'].dropna().to_frame()

stock_data = stock_data.pct_change().dropna()

data = stock_data

data.head(6)


[*********************100%%**********************]  1 of 1 completed


,Close
Date,
2014-09-18,-0.071926
2014-09-19,-0.069843
2014-09-20,0.035735
2014-09-21,-0.024659
2014-09-22,0.008352
2014-09-23,0.083647


In [24]:
# Classifying Changes into States
def classify_change(x):
    if x > 0.06:
        return 'Significant Increase'
    elif x >= 0.02:
        return 'Slight Increase'
    elif x > -0.02:
        return 'No Change'
    elif x >= -0.06:
        return 'Slight Decrease'
    else:
        return 'Significant Decrease'
#Applying Classification
data['State'] = data['Close'].apply(classify_change)
        

In [25]:
data['State'].head()

Date
2014-09-18    Significant Decrease
2014-09-19    Significant Decrease
2014-09-20         Slight Increase
2014-09-21         Slight Decrease
2014-09-22               No Change
Name: State, dtype: object

In [26]:
#Creating Transition Matrix
data['Previous State'] = data['State'].shift(1)

#Creating Transition Matrix
transition_matrix = pd.crosstab(data['Previous State'], data['State'], normalize = 'index')

In [27]:
print(transition_matrix)

State                 No Change  Significant Decrease  Significant Increase  \
Previous State                                                                
No Change              0.652697              0.033719              0.035645   
Significant Decrease   0.408805              0.100629              0.132075   
Significant Increase   0.433526              0.086705              0.098266   
Slight Decrease        0.525773              0.080412              0.057732   
Slight Increase        0.541391              0.029801              0.054636   

State                 Slight Decrease  Slight Increase  
Previous State                                          
No Change                    0.127649         0.150289  
Significant Decrease         0.138365         0.220126  
Significant Increase         0.196532         0.184971  
Slight Decrease              0.129897         0.206186  
Slight Increase              0.167219         0.206954  


In [29]:
#Create n-step transition matrix
def compute_n_step_transition_matrix(matrix, n):
    return np.linalg.matrix_power(matrix, n)

transition_matrix_np = transition_matrix.to_numpy()
n_steps = input("Enter How Many Steps")
n_steps = int(n_steps)
n_step_matrix = compute_n_step_transition_matrix(transition_matrix_np, n_steps)
n_step_matrix_df = pd.DataFrame(n_step_matrix, index=transition_matrix.index, columns=transition_matrix.columns)

# Print the n-step transition matrix
print(n_step_matrix_df)

Enter How Many Steps 1


State                 No Change  Significant Decrease  Significant Increase  \
Previous State                                                                
No Change              0.652697              0.033719              0.035645   
Significant Decrease   0.408805              0.100629              0.132075   
Significant Increase   0.433526              0.086705              0.098266   
Slight Decrease        0.525773              0.080412              0.057732   
Slight Increase        0.541391              0.029801              0.054636   

State                 Slight Decrease  Slight Increase  
Previous State                                          
No Change                    0.127649         0.150289  
Significant Decrease         0.138365         0.220126  
Significant Increase         0.196532         0.184971  
Slight Decrease              0.129897         0.206186  
Slight Increase              0.167219         0.206954  
